## UPD con Python 

El protocolo de datagramas de usuario (UDP) funciona de forma diferente al TCP/IP. Mientras que TCP es un protocolo orientado a la corriente, que asegura que todos los datos se transmitan en el orden correcto, el UDP es un protocolo orientado a los mensajes. El UDP no requiere una conexión de larga duración, por lo que la configuración de una toma UDP es un poco más simple. 

Por otra parte, los mensajes UDP deben caber dentro de un solo paquete (para IPv4, eso significa que sólo pueden contener 65.507 bytes porque el paquete de 65.535 bytes también incluye información de cabecera) y la entrega no está garantizada como ocurre con TCP.

<img src="https://2.bp.blogspot.com/-obs52hAZL-E/WGD8_x9caLI/AAAAAAAAAhI/FgreOO7QK3Mkk8XRRNwo0cdPzLCOJNZXwCPcB/s640/pyt.png">

## Ejemplo Chat 

### Servidor

In [ ]:

import socket

#Definimos la ip de la entrada de mensajes
localIP = "127.0.0.1"

#Escogemos el puerto entre los disponibles 
localPort = 20001

#Definimos el tamano del buffer
bufferSize = 1024

msgFromServer = "Hello UDP Client"

bytesToSend = str.encode(msgFromServer)

# Creamos el datagram socket

UDPServerSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)

# Conectamos la direccion ip con el puerto

UDPServerSocket.bind((localIP, localPort))

print("UDP server up and listening")

# Escuchamos los datagramas que vengan de los clientes

while (True):
    bytesAddressPair = UDPServerSocket.recvfrom(bufferSize)

    message = bytesAddressPair[0]

    address = bytesAddressPair[1]
    #Mensajes de los clientes
    clientMsg = "Message from Client:{}".format(message)
    clientIP = "Client IP Address:{}".format(address)

    print(clientMsg)
    print(clientIP)

    # Mandamos un reply al cliente 

    UDPServerSocket.sendto(bytesToSend, address)

### Cliente

In [ ]:
import socket

msgFromClient = "Hello UDP Server"

bytesToSend = str.encode(msgFromClient)

serverAddressPort = ("127.0.0.1", 20001)

bufferSize = 1024

# Creamos el socket udp por el lado del cliente

UDPClientSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)

# Mandamos el mensaje utilizando el socked de cliente UDP

UDPClientSocket.sendto(bytesToSend, serverAddressPort)

msgFromServer = UDPClientSocket.recvfrom(bufferSize)

msg = "Message from Server {}".format(msgFromServer[0])

print(msg)



## Ejemplo Audio

### Instalacion PyAudio

Se trata de un binding en Python de la conocida librería de audio I/O multi-plataforma y open source PortAudio. El objetivo es capturar una señal de audio a través del micrófono de nuestro ordenador, para luego visualizar su representación temporal y también su representación frecuencial

``pip install -m pyAudio``

En caso de tener problemas con la instalación, es probable que sea por un problema en Wheel que se construye con respecto a tu versión de python. En ese caso miramos nuestra version con el comando de python en consola. Miramos la carpeta de pyaudio en los archivos anexos para obtener las versiones compatibles.

``python``

Les aparecerá algo tal que así

``Python 3.7.3 (default, Apr 24 2019, 15:29:51) [MSC v.1915 64 bit (AMD64)] :: Anaconda, Inc. on win32``

De acuerdo a su versión instalan con el comando pip installa ARCHIVO, indicando el nombre con su versión compatible, en caso de que no funcione la versión intentar con las demas

## Servidor

In [ ]:
import socket
import Pyaudio
from threading import Thread

frames = []

def udpStream():
    udp = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

    while True:
        if len(frames) > 0:
            udp.sendto(frames.pop(0), ("127.0.0.1", 12345)) #
    udp.close()

def record(stream, CHUNK):
    while True:
        frames.append(stream.read(CHUNK))

if __name__ == "__main__":
    CHUNK = 1024
    FORMAT = pyaudio.paInt16 #Decodificacion del audioo
    CHANNELS = 2 #Escoger si es sterio o mono
    RATE = 44100 #El rate del audio

    Audio = pyaudio.PyAudio()

    stream = Audio.open(format = FORMAT,
                    channels = CHANNELS,
                    rate = RATE,
                    input = True,
                    frames_per_buffer = CHUNK,
                    )

#Inicializacion de hilos
    AudioThread = Thread(target = record, args = (stream, CHUNK,))
    udpThread = Thread(target = udpStream)
    AudioThread.setDaemon(True)
    udpThread.setDaemon(True)
    AudioThread.start()
    udpThread.start()
    AudioThread.join()
    udpThread.join()


### Cliente

In [ ]:
import pyaudio
import socket
from threading import Thread

frames = []

def udpStream(CHUNK):

    udp = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    udp.bind(("127.0.0.1", 12345))

    while True:
        soundData, addr = udp.recvfrom(CHUNK*CHANNELS*2)
        frames.append(soundData)
    udp.close()

def play(stream, CHUNK):
    BUFFER = 10
    while True:
            if len(frames) == BUFFER:
                while True:
                    stream.write(frames.pop(0), CHUNK)

if __name__ == "__main__":
    FORMAT = pyaudio.paInt16
    CHUNK = 1024
    CHANNELS = 2
    RATE = 44100

    Audio = pyaudio.PyAudio()

    stream = Audio.open(format=FORMAT,
                    channels = CHANNELS,
                    rate = RATE,
                    output = True,
                    frames_per_buffer = CHUNK,
                    )

    udpThread  = Thread(target = udpStream, args=(CHUNK,))
    AudioThread  = Thread(target = play, args=(stream, CHUNK,))
    udpThread .setDaemon(True)
    AudioThread.setDaemon(True)
    udpThread .start()
    AudioThread.start()
    udpThread .join()
    AudioThread.join()